In [ ]:
# !apt install python3-pip
# !pip3 install ffmpeg
# !pip3 install vosk
# !pip3 install torch
# !pip3 install transformers
# !pip3 install pyaudio
# !pip3 install twisted

In [ ]:
# !curl -o ./model_small.zip https://alphacephei.com/vosk/models/vosk-model-small-ru-0.22.zip
# !unzip model_small.zip
# !mv vosk-model-small-ru-0.22/ model
# rm -rf model_small.zip

In [ ]:
from vosk import Model, KaldiRecognizer
import os
import json
import pyaudio
from twisted.internet.protocol import DatagramProtocol
from twisted.internet import reactor
import socket

In [ ]:
FRAME_RATE = 16000
BUFFER = 1024
CHANNELS=1
multicast_ip = '228.0.0.5'
multicast_port = 9999
model = Model("model")

In [ ]:
def get_ip():
    s = socket.socket(socket.AF_INET, socket.SOCK_DGRAM)
    s.settimeout(0)
    try:
        # doesn't even have to be reachable
        s.connect(('10.254.254.254', 1))
        IP = s.getsockname()[0]
    except Exception:
        IP = '127.0.0.1'
    finally:
        s.close()
    return IP
print(get_ip()) 

In [ ]:
results = []
class Client(DatagramProtocol):
    def startProtocol(self):
        py_audio = pyaudio.PyAudio()
        self.buffer = BUFFER  
        
        if MULT==True:
            self.another_client = multicast_ip, multicast_port
        else:
            self.another_client = input("Write address: "), int(input("Write port: "))
        
        self.rec = KaldiRecognizer(model, FRAME_RATE)
        self.rec.SetWords(True)
        self.outrec = KaldiRecognizer(model, FRAME_RATE)
        self.outrec.SetWords(True)
        
        # группа для мультикаст передачи, так же остается передача напрямую по ip
        self.transport.joinGroup(multicast_ip)

        # выдаст ошибку если нет колонок
        self.output_stream = py_audio.open(format=pyaudio.paInt16,
                                           output=True, rate=FRAME_RATE, channels=CHANNELS,
                                           frames_per_buffer=self.buffer)
        
        # выдаст ошибку если нет микрофона
        self.input_stream = py_audio.open(format=pyaudio.paInt16,
                                          input=True, rate=FRAME_RATE, channels=CHANNELS,
                                          frames_per_buffer=self.buffer)
        reactor.callInThread(self.record)

    # передача голоса по ип либо всем мультикаст
    def record(self):
        while True:
            data = self.input_stream.read(self.buffer,exception_on_overflow=False)
            self.transport.write(data, self.another_client)
            
            self.outrec.AcceptWaveform(data)
            if 'стоп' in self.outrec.PartialResult():
                break

    # прием голоса, преобразование в текст и логирование, воспроизведение если передача не с этого клиента
    def datagramReceived(self, datagram, addr):
        if addr[0]!=get_ip():
            self.output_stream.write(datagram)
            
            if self.rec.AcceptWaveform(datagram):
                res = self.rec.Result()
                print(json.loads(res)['text'],addr)
                results.append((json.loads(res)['text'],addr))

In [ ]:
MULT = True 
# MULT = False
port = multicast_port
#228.0.0.5 9999
reactor.listenMulticast(multicast_port, Client(), listenMultiple=True)
reactor.run()

In [ ]:
results